In [149]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/rg/vtjd9wkj6cq1lck908ywby500000gn/T/ipykernel_3271/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [150]:
import numpy as np
import pandas as pd
import networkx as nx

## Project 1. Due by Thursday, February 13. ##

For this project we will use the **London Underground Data**.

1. Compute a Minimum Spanning Tree using, as costs, the straight-line distances of the provided connections, i.e., length of connections computed using the latitude and longitude of each node.

2. Repeat this computation, now using the length of connections provided in the data.

3. Using version 2 of the lengths, we will investigate the robustness of the computation.  We conduct 10000 experiments.  In each experiment the length of each connection is adjusted as follows:
        modified length = max{ (true length)*(1 + Gaussian(0,0.3)), 0.01*(true length) }
i.e., we modify the length randomly using a zero-norm Gaussian distribution with standard deviation 0.3 and correct if the length becomes too small.

  Plot the distribution of minimum spanning tree lengths, as well as the Value-At-Risk at level 5%.

4. Returning to the lengths as in version 2, conduct a delete-and-repair analysis.  That is to say, for each link L of the minimum spanning tree, we

    (a) Remove L from the network.
    
    (b) Compute a minimum spanning tree for the network with L (and only L) removed.  

  Plot the resulting values for the length of the minimum spanning tree.

Note: we only remove one link at a time.



### IMPORTANT ###
1. Please do not submit data.  If you need to generate additional data, submit the code for that as well.
2. By the due date for the project, please make an appointment with the TAs/CAs.  

### Loading Data
Each csv file was loaded into a Pandas dataframe

In [151]:
# Load data into Pandas Dataframes
connections = pd.read_csv("./London Underground/connections.csv", index_col=0)
lines = pd.read_csv("./London Underground/lines.csv")
stations = pd.read_csv("./London Underground/stations.csv")

In [152]:
connections

,station1,station2,line,time,distance(miles)
0,11,163,1,1.0,0.257855
1,11,212,1,2.0,0.454449
2,49,87,1,1.0,0.111148
3,49,197,1,2.0,0.426744
4,82,163,1,2.0,0.273407
...,...,...,...,...,...
428,326,327,14,NaN,6.740020
429,327,328,14,NaN,4.810656
430,319,330,14,NaN,2.606461
431,330,331,14,NaN,NaN


In [153]:
stations.rename(columns={"Unnamed: 0": "station"}, inplace=True)
stations

,station,latitude,longitude,name,zone,total_lines,rail
0,1,51.502800,-0.280100,Acton Town,3.0,2.0,0.0
1,2,51.514300,-0.075500,Aldgate,1.0,2.0,0.0
2,3,51.515400,-0.072600,Aldgate East,1.0,2.0,0.0
3,4,51.510700,-0.013000,All Saints,2.0,1.0,0.0
4,5,51.540700,-0.299700,Alperton,4.0,1.0,0.0
...,...,...,...,...,...,...,...
327,329,51.503095,-0.420452,Hayes & Harlington,5,NaN,NaN
328,330,51.471318,-0.453178,Heathrow Terminals 2 & 3,6,NaN,NaN
329,331,NaN,NaN,Heathrow Terminals 4,NaN,NaN,NaN
330,332,51.471318,-0.453178,Heathrow Terminals 2 & 3,6,NaN,NaN


In [154]:
# Merge connections and stations into one dataframe

stations1 = stations.rename(columns={
    "station": "station1",
    "latitude": "st1_latitude",
    "longitude": "st1_longitude",
    "name": "st1_station_name",
    "zone": "st1_zone",
    "total_lines": "st1_total_lines",
    "rail": "st1_rail"
})
stations2 = stations.rename(columns={
    "station": "station2",
    "latitude": "st2_latitude",
    "longitude": "st2_longitude",
    "name": "st2_station_name",
    "zone": "st2_zone",
    "total_lines": "st2_total_lines",
    "rail": "st2_rail"
})

df = connections.merge(stations1, on="station1", how="left")
df = df.merge(stations2, on="station2",how="left")

df

,station1,station2,line,time,distance(miles),st1_latitude,st1_longitude,st1_station_name,st1_zone,st1_total_lines,st1_rail,st2_latitude,st2_longitude,st2_station_name,st2_zone,st2_total_lines,st2_rail
0,11,163,1,1.0,0.257855,51.522600,-0.157100,Baker Street,1.0,5.0,0.0,51.522500,-0.163100,Marylebone,1.0,1.0,1.0
1,11,212,1,2.0,0.454449,51.522600,-0.157100,Baker Street,1.0,5.0,0.0,51.523400,-0.146600,Regent's Park,1.0,1.0,0.0
2,49,87,1,1.0,0.111148,51.508000,-0.124700,Charing Cross,1.0,2.0,1.0,51.507400,-0.122300,Embankment,1.0,4.0,0.0
3,49,197,1,2.0,0.426744,51.508000,-0.124700,Charing Cross,1.0,2.0,1.0,51.509800,-0.134200,Picadilly Circus,1.0,2.0,0.0
4,82,163,1,2.0,0.273407,51.519900,-0.167900,Edgware Road (B),1.0,1.0,0.0,51.522500,-0.163100,Marylebone,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,326,327,14,NaN,6.740020,51.518538,-0.722533,Maidenhead,Outside,NaN,NaN,51.475462,-0.863242,Twyford,Outside,NaN,NaN
429,327,328,14,NaN,4.810656,51.475462,-0.863242,Twyford,Outside,NaN,NaN,51.458688,-0.971795,Reading,Outside,NaN,NaN
430,319,330,14,NaN,2.606461,51.503095,-0.420452,Hayes & Harlington,5,NaN,NaN,51.471318,-0.453178,Heathrow Terminals 2 & 3,6,NaN,NaN
431,330,331,14,NaN,NaN,51.471318,-0.453178,Heathrow Terminals 2 & 3,6,NaN,NaN,NaN,NaN,Heathrow Terminals 4,NaN,NaN,NaN


In [155]:
df = df.dropna()

### 1. Compute a Minimum Spanning Tree using, as costs, the straight-line distances of the provided connections, i.e., length of connections computed using the latitude and longitude of each node.

In [156]:
# A function to calculate the straight-line distances of the provided connections
def straightLineDistance(row):
    lat1, long1 = row["st1_latitude"], row["st1_longitude"]
    lat2, long2 = row["st2_latitude"], row["st2_latitude"]
    distance = np.sqrt((lat2-lat1)**2 + (long2-long1)**2)
    return distance

In [157]:
df["straight_line_distance"] = df.apply(straightLineDistance, axis=1)
df

/var/folders/rg/vtjd9wkj6cq1lck908ywby500000gn/T/ipykernel_3271/1779691174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["straight_line_distance"] = df.apply(straightLineDistance, axis=1)


,station1,station2,line,time,distance(miles),st1_latitude,st1_longitude,st1_station_name,st1_zone,st1_total_lines,st1_rail,st2_latitude,st2_longitude,st2_station_name,st2_zone,st2_total_lines,st2_rail,straight_line_distance
0,11,163,1,1.0,0.257855,51.5226,-0.1571,Baker Street,1.0,5.0,0.0,51.5225,-0.1631,Marylebone,1.0,1.0,1.0,51.679600
1,11,212,1,2.0,0.454449,51.5226,-0.1571,Baker Street,1.0,5.0,0.0,51.5234,-0.1466,Regent's Park,1.0,1.0,0.0,51.680500
2,49,87,1,1.0,0.111148,51.5080,-0.1247,Charing Cross,1.0,2.0,1.0,51.5074,-0.1223,Embankment,1.0,4.0,0.0,51.632100
3,49,197,1,2.0,0.426744,51.5080,-0.1247,Charing Cross,1.0,2.0,1.0,51.5098,-0.1342,Picadilly Circus,1.0,2.0,0.0,51.634500
4,82,163,1,2.0,0.273407,51.5199,-0.1679,Edgware Road (B),1.0,1.0,0.0,51.5225,-0.1631,Marylebone,1.0,1.0,1.0,51.690400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,198,272,11,1.0,0.412421,51.4893,-0.1334,Pimlico,1.0,1.0,0.0,51.4861,-0.1253,Vauxhall,1.5,1.0,1.0,51.619500
402,198,273,11,3.0,0.695057,51.4893,-0.1334,Pimlico,1.0,1.0,0.0,51.4965,-0.1447,Victoria,1.0,3.0,1.0,51.629901
403,224,260,11,3.0,0.783457,51.5822,-0.0749,Seven Sisters,3.0,1.0,1.0,51.5882,-0.0594,Tottenham Hale,3.0,1.0,1.0,51.663100
404,245,272,11,3.0,0.957943,51.4723,-0.1230,Stockwell,2.0,2.0,0.0,51.4861,-0.1253,Vauxhall,1.5,1.0,1.0,51.609102


In [158]:
df_kruskal = df[["station1", "station2", "straight_line_distance"]].copy()
df_kruskal

,station1,station2,straight_line_distance
0,11,163,51.679600
1,11,212,51.680500
2,49,87,51.632100
3,49,197,51.634500
4,82,163,51.690400
...,...,...,...
401,198,272,51.619500
402,198,273,51.629901
403,224,260,51.663100
404,245,272,51.609102


We decided to implement Kruskal's algorithm to determine the minimum spanning tree using the straight-line distances as weights.
Below code is from Geeks for Geeks

In [159]:
# Class to represent a graph
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = []

    # Function to add an edge to graph
    def addEdge(self, u, v, w):
        self.graph.append([u, v, w])

    # A utility function to find set of an element i
    # (truly uses path compression technique)
    def find(self, parent, i):
        if parent[i] != i:

            # Reassignment of node's parent
            # to root node as
            # path compression requires
            parent[i] = self.find(parent, parent[i])
        return parent[i]

    # A function that does union of two sets of x and y
    # (uses union by rank)
    def union(self, parent, rank, x, y):

        # Attach smaller rank tree under root of
        # high rank tree (Union by Rank)
        if rank[x] < rank[y]:
            parent[x] = y
        elif rank[x] > rank[y]:
            parent[y] = x

        # If ranks are same, then make one as root
        # and increment its rank by one
        else:
            parent[y] = x
            rank[x] += 1

    # The main function to construct MST
    # using Kruskal's algorithm
    def KruskalMST(self):

        # This will store the resultant MST
        result = []

        # An index variable, used for sorted edges
        i = 0

        # An index variable, used for result[]
        e = 0

        # Sort all the edges in
        # non-decreasing order of their
        # weight
        self.graph = sorted(self.graph,
                            key=lambda item: item[2])

        parent = []
        rank = []

        # Create V subsets with single elements
        for node in range(self.V):
            parent.append(node)
            rank.append(0)

        # Number of edges to be taken is less than to V-1
        while e < self.V - 1:

            # Pick the smallest edge and increment
            # the index for next iteration
            u, v, w = self.graph[i]
            i = i + 1
            x = self.find(parent, u)
            y = self.find(parent, v)

            # If including this edge doesn't
            # cause cycle, then include it in result
            # and increment the index of result
            # for next edge
            if x != y:
                e = e + 1
                result.append([u, v, w])
                self.union(parent, rank, x, y)
            # Else discard the edge

        minimumCost = 0
        print("Edges in the constructed MST")
        for u, v, weight in result:
            minimumCost += weight
            print("%d -- %d == %d" % (u, v, weight))
        print("Minimum Spanning Tree", minimumCost)

In [163]:
g = Graph(len(stations))

edges = [(int(u), int(v), float(w)) for u, v, w in df_kruskal.values.tolist()]

for u, v, w in edges:
    g.addEdge(u, v, w)

# Run Kruskal's MST
# g.KruskalMST()


Here, we found out that the station IDs are not continuous. There is no station 189! (Station numbers start from 1)
So we decided to create a mapping of index (index of stations.csv) to station numbers.
We grab the number of unique_stations and use them as the number of vertices to construct the graph.

In [164]:
unique_stations = set(df_kruskal["station1"]).union(set(df_kruskal["station2"]))
station_id_map = {station: idx for idx, station in enumerate(sorted(unique_stations))}

g = Graph(len(unique_stations))

edges = [(station_id_map[u], station_id_map[v], float(w)) for u, v, w in df_kruskal.values.tolist()]

for u, v, w in edges:
    g.addEdge(u, v, w)
g.KruskalMST()


Edges in the constructed MST
265 -- 266 == 51
128 -- 266 == 51
84 -- 128 == 51
65 -- 66 == 51
65 -- 84 == 51
20 -- 267 == 51
20 -- 66 == 51
64 -- 96 == 51
18 -- 96 == 51
19 -- 215 == 51
19 -- 64 == 51
14 -- 267 == 51
14 -- 77 == 51
201 -- 215 == 51
62 -- 217 == 51
62 -- 201 == 51
85 -- 151 == 51
77 -- 268 == 51
15 -- 172 == 51
63 -- 105 == 51
68 -- 85 == 51
42 -- 182 == 51
63 -- 134 == 51
68 -- 105 == 51
134 -- 170 == 51
42 -- 217 == 51
42 -- 78 == 51
60 -- 170 == 51
15 -- 90 == 51
97 -- 172 == 51
97 -- 209 == 51
102 -- 108 == 51
198 -- 287 == 51
90 -- 108 == 51
26 -- 199 == 51
60 -- 236 == 51
26 -- 78 == 51
198 -- 268 == 51
42 -- 287 == 51
3 -- 199 == 51
41 -- 182 == 51
119 -- 236 == 51
41 -- 119 == 51
199 -- 290 == 51
173 -- 251 == 51
199 -- 282 == 51
209 -- 273 == 51
245 -- 287 == 51
174 -- 251 == 51
3 -- 69 == 51
50 -- 102 == 51
35 -- 287 == 51
50 -- 213 == 51
31 -- 69 == 51
40 -- 251 == 51
152 -- 245 == 51
154 -- 282 == 51
32 -- 35 == 51
32 -- 163 == 51
40 -- 214 == 51
154 -- 223 

2. Repeat this computation, now using the length of connections provided in the data.

In [165]:
df_kruskal_2 = df[["station1", "station2", "distance(miles)"]].copy()
df_kruskal_2

,station1,station2,distance(miles)
0,11,163,0.257855
1,11,212,0.454449
2,49,87,0.111148
3,49,197,0.426744
4,82,163,0.273407
...,...,...,...
401,198,272,0.412421
402,198,273,0.695057
403,224,260,0.783457
404,245,272,0.957943


In [166]:
unique_stations = set(df_kruskal_2["station1"]).union(set(df_kruskal_2["station2"]))
station_id_map = {station: idx for idx, station in enumerate(sorted(unique_stations))}

g2 = Graph(len(unique_stations))

edges = [(station_id_map[u], station_id_map[v], float(w)) for u, v, w in df_kruskal_2.values.tolist()]

for u, v, w in edges:
    g2.addEdge(u, v, w)
g2.KruskalMST()

Edges in the constructed MST
48 -- 86 == 0
41 -- 119 == 0
41 -- 290 == 0
199 -- 290 == 0
43 -- 160 == 0
43 -- 165 == 0
59 -- 150 == 0
119 -- 236 == 0
26 -- 78 == 0
40 -- 214 == 0
10 -- 162 == 0
62 -- 201 == 0
214 -- 274 == 0
48 -- 150 == 0
81 -- 162 == 0
155 -- 166 == 0
100 -- 225 == 0
3 -- 199 == 0
150 -- 195 == 0
85 -- 151 == 0
134 -- 170 == 0
63 -- 134 == 0
1 -- 261 == 0
88 -- 275 == 0
13 -- 91 == 0
16 -- 291 == 0
101 -- 257 == 0
175 -- 232 == 0
282 -- 290 == 0
101 -- 275 == 0
147 -- 277 == 0
64 -- 96 == 0
73 -- 285 == 0
27 -- 190 == 0
132 -- 145 == 0
40 -- 251 == 0
146 -- 149 == 0
59 -- 125 == 0
246 -- 283 == 0
19 -- 215 == 0
12 -- 166 == 0
68 -- 85 == 0
89 -- 103 == 0
31 -- 69 == 0
240 -- 263 == 0
1 -- 155 == 0
60 -- 170 == 0
150 -- 257 == 0
39 -- 169 == 0
12 -- 248 == 0
19 -- 64 == 0
9 -- 94 == 0
47 -- 125 == 0
32 -- 163 == 0
63 -- 105 == 0
24 -- 160 == 0
98 -- 234 == 0
93 -- 252 == 0
38 -- 127 == 0
218 -- 220 == 0
185 -- 206 == 0
27 -- 161 == 0
60 -- 236 == 0
68 -- 105 == 0
73 -